In [ ]:
import pandas as pd
import os
import re
import numpy as np
from pathlib import Path

In [ ]:
OUTPUT_DIR = "/h/haoran/projects/HurtfulWords/data/"
HIDE_CIs = True

In [ ]:
sheets = ['overall', 'gender' ,'language', 'insurance', 'ethnicity'] 
allowed_models = {'overall': ['debiased_gender','baseline'],
                 'gender': ['debiased_gender','baseline'],
                 'language': ['baseline'],
                 'insurance': ['baseline'],
                 'ethnicity': ['baseline']}

In [ ]:
def gap_significant(lower, upper):
    return (((lower <0) & (upper < 0)) | ((lower >0) & (upper > 0)))

In [ ]:
dfs = {}
for sheet in sheets:
    res = pd.DataFrame()
    for subdir, dirs, files in tqdm(os.walk(os.path.join(OUTPUT_DIR, 'models', 'finetuned'))):
        for file in files:
            if file.endswith('.xlsx') and not(file.startswith('~')):
                fname = os.path.basename(subdir)
                if 'adv_clinical' in fname:
                    model = 'debiased_gender'
                else:
                    model = 'baseline'
                if model not in allowed_models[sheet]:
                    break

                if 'inhosp_mort' in fname:
                    target = 'inhosp_mort'
                else:
                    name = re.findall(r'.*512_(.*)$', fname)[0]
                    if 'phenotype_all' in fname:
                        target = 'phenotype_all_%s' % name
                    else:
                        target = 'phenotype_first_%s' % name

                if sheet == 'overall':
                    df = pd.read_excel(os.path.join(subdir, file), index_col=0)
                    columns = ['all_auroc', 'all_auprc','all_specificity', 'all_recall', 'all_nsamples']
                    for i in columns:
                        res.loc[target, model + '_' +i] = df.loc[i, 'avg']
                        if not HIDE_CIs:
                            res.loc[target, model + '_' +i+'lowerCI'] = df.loc[i, '2.5%']
                            res.loc[target, model + '_' +i+'upperCI'] = df.loc[i, '97.5%']

                elif sheet == 'gender':
                    df = pd.read_excel(os.path.join(subdir, file), index_col=0, sheet_name = 'gender')
                    
                    columns = {
                            'gender=="F"_dgap_max':'Parity Gap (F-M)',
                               'gender=="F"_egap_positive_max':'Recall Gap',
                               'gender=="F"_egap_negative_max':'Specificity Gap',
                              }
                    for i in columns:
                        res.loc[target, model + '_' +columns[i]] = df.loc[i, 'avg']
                        if not HIDE_CIs:
                            res.loc[target, model + '_' +columns[i]+'lowerCI'] = df.loc[i, '2.5%']
                            res.loc[target, model + '_' +columns[i]+'upperCI'] = df.loc[i, '97.5%']
                        n = i.replace('gender==', '')
                        if n[4:8] in ('egap', 'dgap'):
                            res.loc[target, model + '_' +columns[i]+'sig'] = gap_significant(df.loc[i, '2.5%'], df.loc[i, '97.5%'])                 
                    
                elif sheet == 'insurance':                            
                    df = pd.read_excel(os.path.join(subdir, file), index_col=0, sheet_name = 'insurance')
                    columns = []
                    for i in ['Medicare', 'Private', 'Medicaid']:
                        for j in ['precision', 'recall', 'specificity', 'pred_prevalence', 'actual_prevalence',
                                 'nsamples', 'dgap_max', 'egap_positive_max', 'egap_negative_max']:
                            columns.append(
                                'insurance=="%s"_%s'%(i,j)
                            )
                    for i in columns:
                        n = i.replace('insurance==', '')
                        res.loc[target, n] = df.loc[i, 'avg']
                        if not HIDE_CIs:
                            res.loc[target, n+'lowerCI'] = df.loc[i, '2.5%']
                            res.loc[target, n+'upperCI'] = df.loc[i, '97.5%']
                        if n[:4] in ['dgap', 'egap']:
                            res.loc[target, n+'sig'] = gap_significant(df.loc[i, '2.5%'], df.loc[i, '97.5%'])

                elif sheet == 'language':
                    df = pd.read_excel(os.path.join(subdir, file), index_col=0, sheet_name = 'language_to_use')
                    columns = {'language_to_use=="English"_dgap_max' : 'Parity Gap (E-O)', 
                               'language_to_use=="English"_egap_positive_max' : 'Recall Gap', 
                               'language_to_use=="English"_egap_negative_max' : 'Specificity Gap' }
                    for i in columns:
                        res.loc[target, columns[i]] = df.loc[i, 'avg']
                        if not HIDE_CIs:
                            res.loc[target, columns[i]+'lowerCI'] = df.loc[i, '2.5%']
                            res.loc[target, columns[i]+'upperCI'] = df.loc[i, '97.5%']
                        res.loc[target, columns[i]+'sig'] = gap_significant(df.loc[i, '2.5%'], df.loc[i, '97.5%'])
                        
                elif sheet == 'ethnicity':
                    df = pd.read_excel(os.path.join(subdir, file), index_col=0, sheet_name = 'ethnicity_to_use')
                    columns = []
                    for i in ['WHITE', 'BLACK', 'ASIAN', 'HISPANIC/LATINO', 'OTHER']:
                        for j in ['precision', 'recall', 'specificity', 'pred_prevalence', 'actual_prevalence',
                                 'nsamples', 'dgap_max', 'egap_positive_max', 'egap_negative_max']:
                            columns.append(
                                'ethnicity_to_use=="%s"_%s'%(i,j)
                            )
                    for i in columns:
                        n = i.replace('ethnicity_to_use==', '')
                        res.loc[target, n] = df.loc[i, 'avg']
                        if not HIDE_CIs:
                            res.loc[target, n+'lowerCI'] = df.loc[i, '2.5%']
                            res.loc[target, n+'upperCI'] = df.loc[i, '97.5%']
                        if re.match(r'"[A-Z]+"_(egap|dgap)(.*)', n):
                            res.loc[target, n+'sig'] = gap_significant(df.loc[i, '2.5%'], df.loc[i, '97.5%'])
    # compute AUPRC differences between debiased and baseline model
    if sheet == 'overall':
        res['AUPRC_gap'] = res['debiased_gender_all_auprc']  - res['baseline_all_auprc']  
    
    # compute disparity differences between debiased and baseline model 
    if sheet == 'gender':        
        res['Parity Dif'] = (res['%s_Parity Gap (F-M)'%'baseline'] - res['%s_Parity Gap (F-M)'%'debiased_gender']).abs()
        res['Recall Dif'] = (res['%s_Recall Gap'%'baseline'] - res['%s_Recall Gap'%'debiased_gender']).abs()
        res['Specificity Dif'] = (res['%s_Specificity Gap'%'baseline'] - res['%s_Specificity Gap'%'debiased_gender']).abs()
        
    
    dfs[sheet] = res.sort_index()

In [ ]:
with pd.ExcelWriter(os.path.join(OUTPUT_DIR, 'all_results.xlsx')) as writer:
    for sheet, df in dfs.items():
        df.to_excel(writer, sheet_name = sheet)